In [61]:
from transformers import CamembertTokenizer, CamembertForTokenClassification
import torch
from datasets import load_dataset
from datasets import Dataset
import pandas as pd
import warnings
from sklearn.preprocessing import LabelEncoder


warnings.filterwarnings("ignore")
df = pd.read_csv('../../data/reservation-first-dataset-train.csv')
#dataset = load_dataset('csv', data_files={'train': '../../data/reservation-first-dataset-train.csv', 'test': '../../data/reservation-first-dataset-test.csv'})
print(df.head())

                                              Phrase                 Départ  \
0  montre-moi les trains dimanche allant de Jarvi...  Jarville-la-Malgrange   
1  quels trains voyagent d'Alençon à Corbeil-Esso...                Alençon   
2  montre-moi les trains pour Saint-Avold depuis ...               Xertigny   
3  montrer les trains de Gargan à Valdahon Camp M...                 Gargan   
4  quels trains sont disponibles de Montbard à Sa...               Montbard   

                   Arrivée  
0      La Bassée-Violaines  
1         Corbeil-Essonnes  
2              Saint-Avold  
3  Valdahon Camp Militaire  
4      Saint-Romain-le-Puy  


Préparation des labels pour la classification

In [59]:

def prepare_labels(sentence, departure, arrival):
    # Tokenisation de la phrase en sous-tokens
    token_strs = sentence.split()

    print(token_strs)
    labels = []
    is_departure = False
    is_arrival = False
    # Parcourir chaque token
    for i, token in enumerate(token_strs):
        print("Token: ", token)
    # Vérifier si le token contient une partie de la ville de départ
    if departure in token:
        if not is_departure:
            labels.append('B-START')  # Premier token de départ
            is_departure = True
        else:
            labels.append('I-START')  # Sous-token suivant du départ
            # Vérifier si le token contient une partie de la ville d'arrivée
    elif arrival in token:
        if not is_arrival:
            labels.append('B-END')  # Premier token de l'arrivée
            is_arrival = True
        else:
            labels.append('I-END')  # Sous-token suivant de l'arrivée
    else:
        labels.append('O')  # Pas d'entité
        print("Non encodés: ", labels)
        
        # Encoder les labels textuels en valeurs numériques
        label_encoder = LabelEncoder()
        encoded_labels = label_encoder.fit_transform(labels)

    return encoded_labels   





In [60]:
# Tokenisation et préparation des labels
tokenizer = CamembertTokenizer.from_pretrained('camembert-base', do_lower_case=True)
train_data = []

# Boucle sur les lignes du DataFrame
for i, row in df.iterrows():
    if i < 5:  # Limiter à 5 lignes pour le test
        sentence = row['Phrase']
        departure = row['Départ']
        arrival = row['Arrivée']
    
        print("La phrase: ", sentence)
        # Tokeniser la phrase
        tokenized_sentence = tokenizer(sentence , truncation=True, padding='max_length', max_length=36)
        tokens = tokenized_sentence['input_ids']
        
        # Préparer les labels
        labels = prepare_labels(sentence, departure, arrival)
    
        # Afficher les tokens et les labels
        print("Tokens: ", tokenizer.convert_ids_to_tokens(tokens))  # Afficher les tokens
        print("Labels (encoded): ", labels)  # Afficher les labels encodés
    
        # Ajouter les données au jeu d'entraînement
        train_data.append({
            "input_ids": tokenized_sentence["input_ids"],
            "attention_mask": tokenized_sentence["attention_mask"],
            "labels": labels
        })
        

# Conversion en Dataset
train_dataset = Dataset.from_pandas(pd.DataFrame(train_data))



La phrase:  montre-moi les trains dimanche allant de Jarville-la-Malgrange à La Bassée-Violaines en première classe sans correspondance partant l'après midi
['montre-moi', 'les', 'trains', 'dimanche', 'allant', 'de', 'Jarville-la-Malgrange', 'à', 'La', 'Bassée-Violaines', 'en', 'première', 'classe', 'sans', 'correspondance', 'partant', "l'après", 'midi']
Token:  montre-moi
Token:  les
Token:  trains
Token:  dimanche
Token:  allant
Token:  de
Token:  Jarville-la-Malgrange
Token:  à
Token:  La
Token:  Bassée-Violaines
Token:  en
Token:  première
Token:  classe
Token:  sans
Token:  correspondance
Token:  partant
Token:  l'après
Token:  midi
Non encodés:  ['O']
Tokens:  ['<s>', '▁montre', '-', 'moi', '▁les', '▁trains', '▁dimanche', '▁allant', '▁de', '▁Jar', 'ville', '-', 'la', '-', 'Mal', 'g', 'range', '▁à', '▁La', '▁Bas', 'sée', '-', 'Vi', 'ola', 'ines', '▁en', '▁première', '▁classe', '▁sans', '▁correspondance', '▁partant', '▁l', "'", 'après', '▁midi', '</s>']
Labels (encoded):  [0]
La ph

UnboundLocalError: cannot access local variable 'encoded_labels' where it is not associated with a value